In [ ]:
# -*- coding: utf-8 -*-
"""
═══════════════════════════════════════════════════════════════════════════════
PSID MASTER PIPELINE - COMPLETE DATA PROCESSING
═══════════════════════════════════════════════════════════════════════════════

This script processes PSID data from raw files to analysis-ready datasets.

Research Question:
Does G1 (1968) homeownership affect G2 and G3 outcomes (homeownership, education)?

Pipeline Stages:
1. Load raw data (FIMS + PSID)
2. Create unique person IDs
3. Link generations (G1 → G2 → G3)
4. Extract homeownership (V103: 1=Own, 5=Rent, 8=Other)
5. Harmonize education across waves
6. Build network structure for visualization
7. Export clean datasets

Outputs:
- analysis_ready_data.csv (for regression analysis)
- family_network_nodes.csv (for visualization)
- family_network_edges.csv (for visualization)
- summary_statistics.txt (data quality report)

Author: Your work + Claude's integration
Date: January 2025
═══════════════════════════════════════════════════════════════════════════════
"""

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================

# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

# Paths
DATA_DIR = "/content/drive/MyDrive/DATA/PSID_data"
OUTPUT_DIR = DATA_DIR  # Save outputs in same directory

# Input files
FIMS_FILE = f"{DATA_DIR}/FIMS_Beth.csv"
PSID_FILE = f"{DATA_DIR}/J355167.csv"
LABELS_FILE = f"{DATA_DIR}/J355167_labels.txt"

# Output files
OUTPUT_ANALYSIS = f"{OUTPUT_DIR}/analysis_ready_data.csv"
OUTPUT_NODES = f"{OUTPUT_DIR}/family_network_nodes.csv"
OUTPUT_EDGES = f"{OUTPUT_DIR}/family_network_edges.csv"
OUTPUT_SUMMARY = f"{OUTPUT_DIR}/pipeline_summary.txt"

print("="*80)
print(" PSID MASTER PIPELINE - COMPLETE DATA PROCESSING ".center(80))
print("="*80)
print()

# =============================================================================
# STAGE 1: LOAD DATA
# =============================================================================

print("STAGE 1: LOADING DATA")
print("-"*80)

# Load FIMS
print(f"Loading FIMS from: {FIMS_FILE}")
fims = pd.read_csv(FIMS_FILE)
print(f"✓ Loaded {len(fims):,} rows × {fims.shape[1]} columns")

# Load PSID
print(f"Loading PSID from: {PSID_FILE}")
psid_raw = pd.read_csv(PSID_FILE)
print(f"✓ Loaded {len(psid_raw):,} rows × {psid_raw.shape[1]} columns")

print()

# =============================================================================
# STAGE 2: CREATE PERSON IDs
# =============================================================================

print("STAGE 2: CREATING UNIQUE PERSON IDs")
print("-"*80)

# PSID person_id: ER30001 * 1000 + ER30002
psid_raw['ER30001'] = pd.to_numeric(psid_raw['ER30001'], errors='coerce').astype('Int64')
psid_raw['ER30002'] = pd.to_numeric(psid_raw['ER30002'], errors='coerce').astype('Int64')
psid_raw.insert(0, 'person_id', psid_raw['ER30001'] * 1000 + psid_raw['ER30002'])

n_unique_persons = psid_raw['person_id'].nunique()
print(f"✓ Created person_id for {n_unique_persons:,} unique individuals")

# FIMS IDs
for col in ['G1ID68', 'G1PN', 'G2ID68', 'G2PN']:
    if col in fims.columns:
        fims[col] = pd.to_numeric(fims[col], errors='coerce').astype('Int64')

fims.insert(0, 'grandparent_id', fims['G1ID68'] * 1000 + fims['G1PN'])
fims.insert(1, 'parent_id', fims['G2ID68'] * 1000 + fims['G2PN'])

# Check for G3
if 'G3ID68' in fims.columns and 'G3PN' in fims.columns:
    fims['G3ID68'] = pd.to_numeric(fims['G3ID68'], errors='coerce').astype('Int64')
    fims['G3PN'] = pd.to_numeric(fims['G3PN'], errors='coerce').astype('Int64')
    fims.insert(2, 'child_id', fims['G3ID68'] * 1000 + fims['G3PN'])
    has_g3 = True
else:
    has_g3 = False

n_g1 = fims['grandparent_id'].nunique()
n_g2 = fims[fims['parent_id'].notna()]['parent_id'].nunique()

print(f"✓ G1 (grandparents): {n_g1:,} unique individuals")
print(f"✓ G2 (parents): {n_g2:,} unique individuals")

if has_g3:
    n_g3 = fims[fims['child_id'].notna()]['child_id'].nunique()
    print(f"✓ G3 (children): {n_g3:,} unique individuals")

print()

# =============================================================================
# STAGE 3: EXTRACT HOMEOWNERSHIP (V103)
# =============================================================================

print("STAGE 3: EXTRACTING HOMEOWNERSHIP")
print("-"*80)

# V103 codes: 1=Own, 5=Rent, 8=Other
if 'V103' in psid_raw.columns:
    print("Using V103 for 1968 homeownership")
    print("Codes: 1=Own, 5=Rent, 8=Other")

    # Create binary homeownership indicator
    psid_raw['homeowner_1968'] = psid_raw['V103'].apply(
        lambda x: 1 if x == 1 else (0 if x == 5 else np.nan)
    )

    # Keep the raw V103 for reference
    psid_raw['v103_raw'] = psid_raw['V103']

    # Summary
    v103_dist = psid_raw['V103'].value_counts(dropna=False)
    print(f"\nV103 Distribution:")
    print(f"  Own (1):   {v103_dist.get(1.0, 0):>8,} ({v103_dist.get(1.0, 0)/len(psid_raw)*100:>5.1f}%)")
    print(f"  Rent (5):  {v103_dist.get(5.0, 0):>8,} ({v103_dist.get(5.0, 0)/len(psid_raw)*100:>5.1f}%)")
    print(f"  Other (8): {v103_dist.get(8.0, 0):>8,} ({v103_dist.get(8.0, 0)/len(psid_raw)*100:>5.1f}%)")
    print(f"  Missing:   {psid_raw['V103'].isna().sum():>8,} ({psid_raw['V103'].isna().sum()/len(psid_raw)*100:>5.1f}%)")

    homeowner_dist = psid_raw['homeowner_1968'].value_counts(dropna=False)
    print(f"\nBinary Homeowner:")
    print(f"  Owner:   {homeowner_dist.get(1, 0):>8,}")
    print(f"  Renter:  {homeowner_dist.get(0, 0):>8,}")
    print(f"  Missing: {psid_raw['homeowner_1968'].isna().sum():>8,}")
else:
    print("⚠ WARNING: V103 not found!")
    psid_raw['homeowner_1968'] = np.nan
    psid_raw['v103_raw'] = np.nan

print()

# =============================================================================
# STAGE 4: HARMONIZE EDUCATION
# =============================================================================

print("STAGE 4: HARMONIZING EDUCATION")
print("-"*80)

# Education variables (most recent first)
EDU_VARS = [
    'ER35152',  # 2023
    'ER34952',  # 2021
    'ER34934',  # 2019
    'ER34930',  # 2019
    'ER34734',  # 2017
    'ER34730',  # 2017
    'ER34534',  # 2015
    'ER34530',  # 2015
    'ER34335',  # 2013
    'ER34331',  # 2013
    'ER33817',  # 2011
    'ER30657',  # 1989
    'ER30584',  # 1985
]

# Check which education variables exist
edu_vars_present = [v for v in EDU_VARS if v in psid_raw.columns]
print(f"Found {len(edu_vars_present)} education variables:")
for var in edu_vars_present:
    n_valid = psid_raw[var].notna().sum()
    print(f"  {var}: {n_valid:>8,} valid values")

# Create harmonized education variable (use most recent non-null)
if edu_vars_present:
    psid_raw['education_years'] = psid_raw[edu_vars_present].bfill(axis=1).iloc[:, 0]

    # Classify into categories
    def classify_education(years):
        """Classify education into interpretable categories"""
        if pd.isna(years):
            return 'Missing'
        elif years < 12:
            return 'HS Dropout'
        elif years == 12:
            return 'HS Grad'
        elif 13 <= years < 16:
            return 'Some College'
        else:
            return 'College Grad'

    psid_raw['education_level'] = psid_raw['education_years'].apply(classify_education)

    # Summary
    print(f"\nHarmonized Education Summary:")
    print(f"  Valid: {psid_raw['education_years'].notna().sum():,}")
    print(f"  Missing: {psid_raw['education_years'].isna().sum():,}")
    print(f"\nEducation Level Distribution:")
    print(psid_raw['education_level'].value_counts())
else:
    print("⚠ WARNING: No education variables found!")
    psid_raw['education_years'] = np.nan
    psid_raw['education_level'] = 'Missing'

print()

# =============================================================================
# STAGE 5: LINK GENERATIONS
# =============================================================================

print("STAGE 5: LINKING GENERATIONS")
print("-"*80)

# Create lookup tables for merging
print("Creating lookup tables...")

# G1 lookup (grandparent attributes)
g1_vars = ['person_id', 'homeowner_1968', 'v103_raw', 'ER32000', 'ER32006']
g1_vars = [v for v in g1_vars if v in psid_raw.columns]
g1_lookup = psid_raw[g1_vars].drop_duplicates('person_id').copy()
g1_lookup.columns = ['grandparent_id'] + [f'g1_{c}' for c in g1_lookup.columns[1:]]

# G2/G3 lookup (education and demographics)
person_vars = ['person_id', 'education_years', 'education_level', 'ER32000', 'ER32006']
person_vars = [v for v in person_vars if v in psid_raw.columns]
person_lookup = psid_raw[person_vars].drop_duplicates('person_id').copy()

# Start with FIMS as the base (all family links)
merged = fims[['grandparent_id', 'parent_id']].dropna().drop_duplicates().copy()
print(f"Starting with {len(merged):,} G1→G2 links")

# Merge G1 attributes
merged = merged.merge(g1_lookup, on='grandparent_id', how='left')
print(f"✓ Merged G1 attributes")

# Merge G2 attributes
g2_lookup = person_lookup.copy()
g2_lookup.columns = ['parent_id'] + [f'g2_{c}' for c in g2_lookup.columns[1:]]
merged = merged.merge(g2_lookup, on='parent_id', how='left')
print(f"✓ Merged G2 attributes")

# Add G3 if available
if has_g3:
    g2_g3_links = fims[['parent_id', 'child_id']].dropna().drop_duplicates()

    # Create a separate G2→G3 dataset
    g2_g3_merged = g2_g3_links.copy()

    # Add G2 attributes (parent info)
    g2_g3_merged = g2_g3_merged.merge(g2_lookup, on='parent_id', how='left')

    # Add G3 attributes
    g3_lookup = person_lookup.copy()
    g3_lookup.columns = ['child_id'] + [f'g3_{c}' for c in g3_lookup.columns[1:]]
    g2_g3_merged = g2_g3_merged.merge(g3_lookup, on='child_id', how='left')

    # Add grandparent link (G1→G2→G3)
    g1_g2_for_g3 = merged[['grandparent_id', 'parent_id', 'g1_homeowner_1968']].drop_duplicates()
    g2_g3_merged = g2_g3_merged.merge(g1_g2_for_g3, on='parent_id', how='left')

    print(f"✓ Created G2→G3 dataset with {len(g2_g3_merged):,} links")

print(f"\nFinal merged data: {len(merged):,} rows")
print()

# =============================================================================
# STAGE 6: CREATE ANALYSIS-READY DATASETS
# =============================================================================

print("STAGE 6: CREATING ANALYSIS-READY DATASETS")
print("-"*80)

# Dataset 1: G1→G2 Analysis (Does G1 homeownership predict G2 education?)
analysis_g1_g2 = merged.copy()

# Add birth year if available (for cohort controls)
if 'ER30004' in psid_raw.columns:
    birth_lookup = psid_raw[['person_id', 'ER30004']].copy()
    birth_lookup.columns = ['parent_id', 'g2_birth_year']
    analysis_g1_g2 = analysis_g1_g2.merge(birth_lookup, on='parent_id', how='left')
    # Calculate birth decade
    analysis_g1_g2['g2_birth_decade'] = (analysis_g1_g2['g2_birth_year'] // 10) * 10

# Clean column names for analysis
analysis_g1_g2.rename(columns={
    'g1_homeowner_1968': 'parent_homeowner',
    'g2_education_years': 'child_education_years',
    'g2_education_level': 'child_education_level',
    'g2_ER32000': 'child_sex',
    'g2_ER32006': 'child_race',
    'g1_ER32000': 'parent_sex',
    'g1_ER32006': 'parent_race'
}, inplace=True)

# Remove rows with missing key variables
analysis_g1_g2_clean = analysis_g1_g2[
    analysis_g1_g2['parent_homeowner'].notna() &
    analysis_g1_g2['child_education_years'].notna()
].copy()

print(f"G1→G2 Analysis Dataset:")
print(f"  Total rows: {len(analysis_g1_g2):,}")
print(f"  Complete cases: {len(analysis_g1_g2_clean):,}")
print(f"  Homeowners: {(analysis_g1_g2_clean['parent_homeowner']==1).sum():,}")
print(f"  Renters: {(analysis_g1_g2_clean['parent_homeowner']==0).sum():,}")

# Dataset 2: G1→G2→G3 Analysis (if G3 exists)
if has_g3:
    analysis_g1_g2_g3 = g2_g3_merged.copy()

    analysis_g1_g2_g3.rename(columns={
        'g1_homeowner_1968': 'grandparent_homeowner',
        'g2_education_years': 'parent_education_years',
        'g2_education_level': 'parent_education_level',
        'g3_education_years': 'child_education_years',
        'g3_education_level': 'child_education_level',
        'g3_ER32000': 'child_sex',
        'g3_ER32006': 'child_race'
    }, inplace=True)

    analysis_g1_g2_g3_clean = analysis_g1_g2_g3[
        analysis_g1_g2_g3['grandparent_homeowner'].notna() &
        analysis_g1_g2_g3['child_education_years'].notna()
    ].copy()

    print(f"\nG1→G2→G3 Analysis Dataset:")
    print(f"  Total rows: {len(analysis_g1_g2_g3):,}")
    print(f"  Complete cases: {len(analysis_g1_g2_g3_clean):,}")

print()

# =============================================================================
# STAGE 7: BUILD NETWORK FOR VISUALIZATION
# =============================================================================

print("STAGE 7: BUILDING NETWORK STRUCTURE")
print("-"*80)

# Create node list (all unique individuals)
nodes_list = []

# G1 nodes
g1_nodes = pd.DataFrame({
    'node_id': fims['grandparent_id'].unique(),
    'generation': 'Gen1'
})
g1_attrs = psid_raw[['person_id', 'homeowner_1968', 'ER32000', 'ER32006']].drop_duplicates('person_id')
g1_nodes = g1_nodes.merge(g1_attrs, left_on='node_id', right_on='person_id', how='left')
g1_nodes.drop(columns=['person_id'], inplace=True)
g1_nodes['education_level'] = 'Unknown'  # G1 education not typically tracked
nodes_list.append(g1_nodes)

# G2 nodes
g2_nodes = pd.DataFrame({
    'node_id': fims['parent_id'].dropna().unique(),
    'generation': 'Gen2'
})
g2_attrs = psid_raw[['person_id', 'education_level', 'ER32000', 'ER32006']].drop_duplicates('person_id')
g2_nodes = g2_nodes.merge(g2_attrs, left_on='node_id', right_on='person_id', how='left')
g2_nodes.drop(columns=['person_id'], inplace=True)
g2_nodes['homeowner_1968'] = np.nan  # G2 homeownership would need separate variable
nodes_list.append(g2_nodes)

# G3 nodes (if available)
if has_g3:
    g3_nodes = pd.DataFrame({
        'node_id': fims['child_id'].dropna().unique(),
        'generation': 'Gen3'
    })
    g3_attrs = psid_raw[['person_id', 'education_level', 'ER32000', 'ER32006']].drop_duplicates('person_id')
    g3_nodes = g3_nodes.merge(g3_attrs, left_on='node_id', right_on='person_id', how='left')
    g3_nodes.drop(columns=['person_id'], inplace=True)
    g3_nodes['homeowner_1968'] = np.nan
    nodes_list.append(g3_nodes)

# Combine all nodes
all_nodes = pd.concat(nodes_list, ignore_index=True)
print(f"Created network nodes: {len(all_nodes):,}")
print(all_nodes['generation'].value_counts())

# Create edge list
edges_list = []

# G1→G2 edges
g1_g2_edges = fims[['grandparent_id', 'parent_id']].dropna().drop_duplicates()
g1_g2_edges.columns = ['source', 'target']
edges_list.append(g1_g2_edges)

# G2→G3 edges
if has_g3:
    g2_g3_edges = fims[['parent_id', 'child_id']].dropna().drop_duplicates()
    g2_g3_edges.columns = ['source', 'target']
    edges_list.append(g2_g3_edges)

# Combine all edges
all_edges = pd.concat(edges_list, ignore_index=True)
print(f"Created network edges: {len(all_edges):,}")

print()

# =============================================================================
# STAGE 8: EXPORT DATASETS
# =============================================================================

print("STAGE 8: EXPORTING DATASETS")
print("-"*80)

# Export analysis-ready data
analysis_g1_g2.to_csv(OUTPUT_ANALYSIS, index=False)
print(f"✓ Saved: {OUTPUT_ANALYSIS}")
print(f"  Rows: {len(analysis_g1_g2):,}")

# Export G1→G2→G3 if exists
if has_g3:
    output_g3 = OUTPUT_ANALYSIS.replace('.csv', '_g1_g2_g3.csv')
    analysis_g1_g2_g3.to_csv(output_g3, index=False)
    print(f"✓ Saved: {output_g3}")
    print(f"  Rows: {len(analysis_g1_g2_g3):,}")

# Export network data for visualization
all_nodes.to_csv(OUTPUT_NODES, index=False)
print(f"✓ Saved: {OUTPUT_NODES}")
print(f"  Nodes: {len(all_nodes):,}")

all_edges.to_csv(OUTPUT_EDGES, index=False)
print(f"✓ Saved: {OUTPUT_EDGES}")
print(f"  Edges: {len(all_edges):,}")

print()

# =============================================================================
# STAGE 9: GENERATE SUMMARY REPORT
# =============================================================================

print("STAGE 9: GENERATING SUMMARY REPORT")
print("-"*80)

summary_lines = []
summary_lines.append("="*80)
summary_lines.append("PSID DATA PROCESSING SUMMARY")
summary_lines.append("="*80)
summary_lines.append("")
summary_lines.append("INPUT FILES:")
summary_lines.append(f"  FIMS: {FIMS_FILE}")
summary_lines.append(f"  PSID: {PSID_FILE}")
summary_lines.append("")
summary_lines.append("DATA QUALITY:")
summary_lines.append(f"  Total individuals: {n_unique_persons:,}")
summary_lines.append(f"  G1 (grandparents): {n_g1:,}")
summary_lines.append(f"  G2 (parents): {n_g2:,}")
if has_g3:
    summary_lines.append(f"  G3 (children): {n_g3:,}")
summary_lines.append("")
summary_lines.append("HOMEOWNERSHIP (V103):")
summary_lines.append(f"  Owners: {(psid_raw['homeowner_1968']==1).sum():,}")
summary_lines.append(f"  Renters: {(psid_raw['homeowner_1968']==0).sum():,}")
summary_lines.append(f"  Missing: {psid_raw['homeowner_1968'].isna().sum():,}")
summary_lines.append("")
summary_lines.append("EDUCATION:")
summary_lines.append(f"  Valid: {psid_raw['education_years'].notna().sum():,}")
summary_lines.append(f"  Missing: {psid_raw['education_years'].isna().sum():,}")
summary_lines.append("")
summary_lines.append("ANALYSIS DATASETS:")
summary_lines.append(f"  G1→G2 complete cases: {len(analysis_g1_g2_clean):,}")
if has_g3:
    summary_lines.append(f"  G1→G2→G3 complete cases: {len(analysis_g1_g2_g3_clean):,}")
summary_lines.append("")
summary_lines.append("OUTPUT FILES:")
summary_lines.append(f"  {OUTPUT_ANALYSIS}")
if has_g3:
    summary_lines.append(f"  {output_g3}")
summary_lines.append(f"  {OUTPUT_NODES}")
summary_lines.append(f"  {OUTPUT_EDGES}")
summary_lines.append("")
summary_lines.append("="*80)

# Save to file
with open(OUTPUT_SUMMARY, 'w') as f:
    f.write('\n'.join(summary_lines))

# Print to console
print('\n'.join(summary_lines))

print(f"\n✓ Saved summary: {OUTPUT_SUMMARY}")
print()

# =============================================================================
# PIPELINE COMPLETE
# =============================================================================

print("="*80)
print(" PIPELINE COMPLETE ".center(80))
print("="*80)
print()
print("✅ All datasets created successfully!")
print()
print("NEXT STEPS:")
print("  1. Review the summary report")
print("  2. Run the analysis script: ANALYSIS_INTERGENERATIONAL_EFFECTS.py")
print("  3. Create visualizations using the network files")
print()
print("="*80)